In [1]:
import os
os.getcwd()

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scanpy as sc
import anndata as ad
sc.settings.verbosity = 0

In [2]:
import GenKI as gk
from GenKI.preprocesing import build_adata
from GenKI.dataLoader import DataLoader
from GenKI.train import VGAE_trainer
from GenKI import utils

%load_ext autoreload
%autoreload 2

/home/g0bi/conda/envs/ogenki/lib/python3.11/site-packages/anndata/utils.py:429: FutureWarning: Importing read_csv from `anndata` is deprecated. Import anndata.io.read_csv instead.
  warnings.warn(msg, FutureWarning)
/home/g0bi/conda/envs/ogenki/lib/python3.11/site-packages/anndata/utils.py:429: FutureWarning: Importing read_excel from `anndata` is deprecated. Import anndata.io.read_excel instead.
  warnings.warn(msg, FutureWarning)
/home/g0bi/conda/envs/ogenki/lib/python3.11/site-packages/anndata/utils.py:429: FutureWarning: Importing read_hdf from `anndata` is deprecated. Import anndata.io.read_hdf instead.
  warnings.warn(msg, FutureWarning)
/home/g0bi/conda/envs/ogenki/lib/python3.11/site-packages/anndata/utils.py:429: FutureWarning: Importing read_loom from `anndata` is deprecated. Import anndata.io.read_loom instead.
  warnings.warn(msg, FutureWarning)
/home/g0bi/conda/envs/ogenki/lib/python3.11/site-packages/anndata/utils.py:429: FutureWarning: Importing read_mtx from `anndata` i

In [3]:
# subset data
adata = build_adata("../data/organoids_SCT.h5ad")
adata = adata[:1200, :3000 ].copy()
adata

load counts from ../data/organoids_SCT.h5ad


,gene_ids,feature_types,mt,ribo,hb,n_cells_by_counts,mean_counts,log1p_mean_counts,pct_dropout_by_counts,total_counts,log1p_total_counts,n_cells,means,variances,residual_variances,highly_variable_rank,highly_variable,mean_expression
WASH7P,ENSG00000227232,Gene Expression,False,False,False,40,0.007489,0.007461,99.334332,45.0,3.828641,40,0.007528,0.009145,1.169127,NaN,False,1.845695e-08
AL627309.1,ENSG00000238009,Gene Expression,False,False,False,5,0.000832,0.000832,99.916791,5.0,1.791759,5,0.000836,0.000836,0.944753,NaN,False,-1.314913e-08
AL627309.6,ENSG00000268903,Gene Expression,False,False,False,7,0.001165,0.001164,99.883508,7.0,2.079442,7,0.001171,0.001170,1.108217,NaN,False,-2.062495e-08
FO538757.1,ENSG00000279457,Gene Expression,False,False,False,7,0.001165,0.001164,99.883508,7.0,2.079442,7,0.001171,0.001170,0.968105,NaN,False,2.237168e-10
AP006222.1,ENSG00000228463,Gene Expression,False,False,False,4,0.000666,0.000665,99.933433,4.0,1.609438,4,0.000669,0.000669,0.759478,NaN,False,-7.479557e-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CYP27A1,ENSG00000135929,Gene Expression,False,False,False,36,0.007489,0.007461,99.400899,45.0,3.828641,36,0.007528,0.011488,1.408993,NaN,False,-2.607329e-09
LINC01494,ENSG00000228135,Gene Expression,False,False,False,4,0.000832,0.000832,99.933433,5.0,1.791759,4,0.000836,0.001170,0.596495,NaN,False,3.848677e-09
AC097468.2,ENSG00000235024,Gene Expression,False,False,False,3,0.000499,0.000499,99.950075,3.0,1.386294,3,0.000502,0.000502,0.584004,NaN,False,-8.435496e-09
CDK5R2,ENSG00000171450,Gene Expression,False,False,False,21,0.004160,0.004152,99.650524,25.0,3.258096,21,0.004182,0.005504,1.254151,NaN,False,-7.845668e-10


In [ ]:
adata.obs

In [ ]:
adata.var

In [ ]:
# load data
data_wrapper =  DataLoader(
                adata, # adata object
                target_gene = ["WASH7P"], # KO gene name
                target_cell = None, # obsname for cell type, if none use all
                obs_label = "gene_ids", # colname for genes
                GRN_file_dir = "GRNs", # folder name for GRNs
                rebuild_GRN = True, # whether build GRN by pcNet
                pcNet_name = "pcNet_organoid", # GRN file name
                verbose = True, # whether verbose
                n_cpus = 8, # multiprocessing
                )

data_wt = data_wrapper.load_data()
data_ko = data_wrapper.load_kodata()

use all the cells (1200) in adata
build GRN


2025-02-21 20:11:59,030	INFO worker.py:1841 -- Started a local Ray instance.


ray init, using 8 CPUs


In [ ]:
# init trainer
hyperparams = {"epochs": 100, 
               "lr": 7e-4, 
               "beta": 1e-4, 
               "seed": 8096}
log_dir=None 

sensei = VGAE_trainer(data_wt, 
                     epochs=hyperparams["epochs"], 
                     lr=hyperparams["lr"], 
                     log_dir=log_dir, 
                     beta=hyperparams["beta"],
                     seed=hyperparams["seed"],
                     verbose=False,
                     )

In [ ]:
# %%timeit
sensei.train()

In [ ]:
sensei.save_model('adata-save')

In [ ]:
# get distance between wt and ko
z_mu_wt, z_std_wt = sensei.get_latent_vars(data_wt)
z_mu_ko, z_std_ko = sensei.get_latent_vars(data_ko)
dis = gk.utils.get_distance(z_mu_ko, z_std_ko, z_mu_wt, z_std_wt, by="KL")
print(dis.shape)

In [ ]:
# raw ranked gene list
res_raw = utils.get_generank(data_wt, dis, rank=True)
res_raw.head()

In [ ]:
# if permutation test
null = sensei.pmt(data_ko, n=100, by="KL")
res = utils.get_generank(data_wt, dis, null,)

# save_significant_as = 'gene_list_example')
res